In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/best-param/best_params.json
/kaggle/input/balanced-yolo-data/data.yaml
/kaggle/input/balanced-yolo-data/valid/labels/image0028348.txt
/kaggle/input/balanced-yolo-data/valid/labels/ffhq_2016.txt
/kaggle/input/balanced-yolo-data/valid/labels/ffhq_5243.txt
/kaggle/input/balanced-yolo-data/valid/labels/image0031450.txt
/kaggle/input/balanced-yolo-data/valid/labels/ffhq_878.txt
/kaggle/input/balanced-yolo-data/valid/labels/ffhq_421.txt
/kaggle/input/balanced-yolo-data/valid/labels/image0029308.txt
/kaggle/input/balanced-yolo-data/valid/labels/image0039337.txt
/kaggle/input/balanced-yolo-data/valid/labels/ffhq_1547.txt
/kaggle/input/balanced-yolo-data/valid/labels/image0041096.txt
/kaggle/input/balanced-yolo-data/valid/labels/ffhq_1406.txt
/kaggle/input/balanced-yolo-data/valid/labels/image0011379.txt
/kaggle/input/balanced-yolo-data/valid/labels/image0015095.txt
/kaggle/input/balanced-yolo-data/valid/labels/ffhq_3060.txt
/kaggle/input/balanced-yolo-data/valid/labels/image00177

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.0/887.0 kB 12.2 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
import cv2
from ultralytics import YOLO
import torch
import warnings
warnings.filterwarnings('ignore')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Load Best Parameter

In [4]:
import json

# Load the best parameters
with open("/kaggle/input/best-param/best_params.json", "r") as f:
    best_trial = json.load(f)

print("Loaded hyperparameters:", best_trial)


Loaded hyperparameters: {'lr0': 0.00010446991982747811, 'momentum': 0.881082406956181, 'weight_decay': 7.273003893562258e-06}


In [5]:

# Extract the best hyperparameters from the best trial
best_lr0 = best_trial['lr0']
best_momentum = best_trial['momentum']
best_weight_decay = best_trial['weight_decay']

# Print the best hyperparameters
print(f"Best hyperparameters:")
print(f"Learning rate: {best_lr0}")
print(f"Momentum: {best_momentum}")
print(f"Weight decay: {best_weight_decay}")

Best hyperparameters:
Learning rate: 0.00010446991982747811
Momentum: 0.881082406956181
Weight decay: 7.273003893562258e-06


# Yolo Trainer Class

In [6]:
from ultralytics import YOLO

class YOLOTrainer:
    def __init__(self, task='', mode='', model='', imgsz=512, data='', plots=True, momentum=None, patience=7,
                 device=None, epochs=None, batch=None, learning_rate=None, optimizer=None, weight_decay=None, name='', exist_ok=True):
        self.task = task
        self.mode = mode
        self.model = model
        self.imgsz = imgsz
        self.data = data
        self.device = device
        self.epochs = epochs
        self.batch = batch
        self.name = name
        self.exist_ok = exist_ok
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.weight_decay = weight_decay
        self.plots = plots
        self.momentum = momentum
        self.patience = patience

    def load_model(self):
        model = YOLO(self.model)  # Load the YOLO model
        return model

    def train(self):
        # Train the model
        model = self.load_model()
        model.train(task=self.task, 
                    mode=self.mode, 
                    data=self.data, 
                    device=self.device,
                    epochs=self.epochs, 
                    batch=self.batch, 
                    imgsz=self.imgsz,
                    name=self.name, 
                    exist_ok=self.exist_ok,  
                    lr0=self.learning_rate,
                    optimizer=self.optimizer, 
                    weight_decay=self.weight_decay,
                    plots=self.plots,
                    momentum=self.momentum,
                    patience=self.patience)
        result = model.val()
        return result.results_dict.get('metrics/mAP50(B)', 0) 
        
    def save_model(self, save_path):
        """Save the trained model to a specified path."""
        if self.trained_model is not None:
            self.trained_model.save(save_path)
            print(f"Model saved to {save_path}")
        else:
            raise ValueError("No trained model found. Please train the model first.")

# Train using Best Parameters

In [7]:
yaml_path = "/kaggle/input/balanced-yolo-data/data.yaml"

In [8]:
# Now, train the model with the best hyperparameters and more epochs
epochs = 100  # You can set this to any number of epochs you want to train for

trainer = YOLOTrainer(
    task='detect', 
    mode='train', 
    model='yolo11m.pt', 
    data=yaml_path,  # Set your dataset config path
    epochs=epochs,  # Train for more epochs
    batch=16, 
    learning_rate=best_lr0,
    momentum=best_momentum,
    optimizer='Adam',  # Can change based on preference
    weight_decay=best_weight_decay, 
    name='final_run',  # A different name for this run
    device='cuda'  # Assuming you're using a GPU, change as needed
)

# Start final training with the best hyperparameters and more epochs
trainer.train()

100%|██████████| 38.8M/38.8M [00:00<00:00, 156MB/s]


Ultralytics 8.3.32 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/kaggle/input/balanced-yolo-data/data.yaml, epochs=100, time=None, patience=7, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=final_run, exist_ok=True, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

100%|██████████| 755k/755k [00:00<00:00, 8.55MB/s]
2024-11-18 15:49:55,142	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-18 15:49:55,641	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 65.2MB/s]


...

val: Scanning /kaggle/input/balanced-yolo-data/valid/labels... 5406 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5406/5406 [00:07<00:00, 723.50it/s]


val: WARNING ⚠️ Cache directory /kaggle/input/balanced-yolo-data/valid is not writeable, cache not saved.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 338/338 [01:45<00:00,  3.20it/s]


                   all       5406       5406      0.661      0.767      0.783      0.783
                 Anger        712        712       0.65      0.795      0.794      0.794
              Contempt        618        618      0.581      0.851      0.786      0.786
               Disgust        672        672      0.642      0.717      0.754      0.754
                  Fear        622        622      0.635      0.811      0.823      0.823
                 Happy        791        791      0.875      0.796      0.921      0.921
               Neutral        514        514      0.557      0.651      0.608      0.608
                   Sad        603        603      0.626      0.766      0.758      0.758
              Surprise        874        874      0.719      0.749      0.819      0.819
Speed: 0.2ms preprocess, 16.0ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/final_run


0.7828383716926484

# Save the model

In [9]:
trainer.save_model("best_model.pt")

AttributeError: 'YOLOTrainer' object has no attribute 'trained_model'